## Thank you for upvoding and comment.
## I got a comment that the flow is easy to understand, so I updated not only the Japanese ver but also the English ver.

In [ ]:
import numpy as np 
import pandas as pd 

# 1. すべてのpathを出す # Get all path

In [ ]:
all_path = []

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        all_path.append(os.path.join(dirname, filename))


In [ ]:
all_path[:10]

In [ ]:
sample_path = '/kaggle/input/landmark-recognition-2020/sample_submission.csv'
traincsv_path = '/kaggle/input/landmark-recognition-2020/train.csv'

In [ ]:
# sample submissionの中身を見る # View contents of sample submission 
sample = pd.read_csv(sample_path)
sample

In [ ]:
# traincsvの中身を見る # View contents of train.csv
traincsv = pd.read_csv(traincsv_path)
traincsv

In [ ]:
# traincsvの中身はidとlandmark_idだということがわかる。 # I could understand that the contents of traincsv are composed of id and landmark_id.
# まずは、idごとにpathをくっつけることを考える。 # Firstly, I considered attaching the path for each id.
# all_pathからtrainが入っているものと、jpgが入っているものを抜く # I tried to extract the one that contained the "train" and ".jpg" from the all_path.

In [ ]:
all_path[:10]

# 2. trainデータの確認と解析 # Confirmation the train data and analyzing.

In [ ]:
# trainフォルダの中の.jpgファイルだけ抜きたい # I wanted to extract only ".jpg" file in train directory.

In [ ]:
# trainがつくやつだけ抜く # Firstly, I extracted only "train" in all path.
train_impath = [s for s in all_path if "train" in s]

In [ ]:
# <わからない人は・・・>↑は以下と同じ。 # If you cannot understand, the above code is the same as below.

train_impath2 = []

for s in all_path:
    if "train" in s:
        train_impath2.append(s)

In [ ]:
# 同様にさらにjpgファイルだけ抜く # After that, I extracted ".jpg" in train_impath.
train_impath = [s for s in train_impath if ".jpg" in s]
train_impath[:10]

In [ ]:
# ここから.jpgの前のidのみを抜きたい  # From these pathes, I extracted id from path in each, in order to merge with train.csv.
train_id = [s.split("/")[-1] for s in train_impath]
train_id = [s.split(".")[0] for s in train_id]
train_id[:10]

In [ ]:
# dataframe化 # Making dataframe.
df = pd.DataFrame()
df["path"] = train_impath
df["id"] = train_id

In [ ]:
df.head(3)

In [ ]:
traincsv.head(3)

In [ ]:
# mergeしてidが同じところをくっつける # Merge with the same id.
traindf = pd.merge(traincsv,df,on="id",how="left")
traindf.head(3)

## landmark id = 1の画像を確認 # Confirmation of images with landmark id = 1.

In [ ]:
import cv2
import matplotlib.pyplot as plt

example = traindf[traindf["landmark_id"]==1]
for a in example["path"]:
    plt.figure()
    img = cv2.imread(a)
    plt.imshow(img)
    plt.axis("off")

## traindfをもう少し解析 # Analyzing the traindf

In [ ]:
traindf.head(3)

In [ ]:
traindf.to_csv("traindf.csv",index=False)

In [ ]:
import collections
l = np.array(traindf["landmark_id"])
c = collections.Counter(l) # get unique counts

In [ ]:
#print(c) # 辞書型で、それぞれのidに対する個数が出ている # c is the type of dictionaly and composed of the id and the number of each id.

In [ ]:
# dataframe化したいので、リスト化する # List the c in order to make the dataframe.
key = list(c.keys())
cnt = list(c.values())

In [ ]:
dfcnt = pd.DataFrame()
dfcnt["id"] = key
dfcnt["count"] = cnt
dfcnt.head(3)

In [ ]:
# 並び替え # sort_values
dfcnt = dfcnt.sort_values("count")
dfcnt = dfcnt.reset_index(drop=True)

In [ ]:
dfcnt.tail(3)

In [ ]:
dfcnt.to_csv("dfcnt.csv",index=False)

In [ ]:
plt.scatter(dfcnt["id"],dfcnt["count"])
plt.xlabel("id",fontsize = 15)
plt.ylabel("count",fontsize = 15)

## id = 138982が6272個で一番多いので、どういう画像か見てみる 
## # Since id = 138982 is the most with 6272, let's see what kind of image.

In [ ]:
import cv2
import matplotlib.pyplot as plt

example = traindf[traindf["landmark_id"]==dfcnt["id"].iloc[-1]]


for a in range(10):
    plt.figure()
    img = cv2.imread(example["path"].iloc[a])
    plt.imshow(img)
    plt.axis("off")

In [ ]:
# 黒い枠ぶちが最も多い画像 # I can understand the images have black borders.

# 2. testデータもみてみる # Confirmation of test data

In [ ]:
# all_pathから抜く # extract the .jpg file path from all_path
test_impath = [s for s in all_path if "test" in s]
test_impath = [s for s in test_impath if ".jpg" in s]
test_impath[:10]

In [ ]:
for a in range(10):
    plt.figure()
    img = cv2.imread(test_impath[a])
    plt.imshow(img)
    plt.axis("off")

In [ ]:
# このテスト画像に訓練画像のidを関連付けてconfidenceを出すのが課題。 # The challenge is to associate the id of the training image with this test image to give confidence.

# 3. 最後にsubmission fileの確認 # Confirmaetion of submission file

In [ ]:
sample

## testデータのid, landmark id, confidence（自信)の順になっている。
## # The order of submission file is the test data id, landmark id, and confidence.

## 例えば、landmarksを一番多いidに全部変えて提出してみる (結果はscore 0だけど、提出可能なフォーマットかの確認。)　
## # For example, I tried changing all the landmarks to the most ids and submit it 
## (the result is score 0, but check if it is in a submittable format).

In [ ]:
dfcnt.tail(3)

In [ ]:
dfcnt["id"].iloc[-1]

In [ ]:
test_landmarks = str(dfcnt["id"].iloc[-1]) + " " + str(1.0)
test_landmarks

In [ ]:
sample.head(3)

In [ ]:
sample["landmarks"] = test_landmarks
sample.head(3)

## 提出名はsubmission.csv　右側のsettingでinternet offにして、commitしないと提出できないので、注意。
## #The file name must be "submission.csv". And it notes that the internet must be off by changing the setting on right part of this screen. 

In [ ]:

sample.to_csv("submission.csv",index=False)